In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
#import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
    #for filename in filenames:
        #print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import joblib
x = joblib.load("/kaggle/input/preprocessing-done/x_data.joblib")
y = joblib.load("/kaggle/input/preprocessing-done/y_data.joblib")
img_ids = joblib.load("/kaggle/input/preprocessing-done/img_ids.joblib")
print(x.shape)
print(y.shape)

(10763, 256, 256, 3)
(10763,)


In [3]:
import numpy as np
from sklearn.model_selection import train_test_split
x1=x[:8000]
y1=y[:8000]
img_ids1=img_ids[:8000]
x_train, x_test, y_train, y_test, img_ids_train, img_ids_test = train_test_split(x1, y1, img_ids1, test_size=0.15, shuffle=True, random_state=42)
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


(6800, 256, 256, 3)
(1200, 256, 256, 3)
(6800,)
(1200,)


In [4]:
import tensorflow as tf
print(tf.test.is_gpu_available())

True


In [5]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dropout
import tensorflow as tf

In [6]:
from tensorflow.keras.models import save_model
num_classes = 6
devices = ['/GPU:0', '/GPU:1']
strategy = tf.distribute.MirroredStrategy(devices=devices)
with strategy.scope(): 
    # 加载预训练的VGG16模型（不包括顶部分类层）
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=(256,256, 3))

    for layer in base_model.layers:
            layer.trainable = False
    # Customize the top classification layers
    x = base_model.output
    x = Flatten()(x)
    x = Dense(4096, activation='relu')(x)
    x = Dropout(0.2)(x)  # 添加 Dropout 层，丢弃比例为 0.2
    x = Dense(4096, activation='relu')(x)
    #x = Dropout(0.2)(x)  # 添加 Dropout 层，丢弃比例为 0.2
    predictions = Dense(num_classes, activation='softmax')(x)  # Replace num_classes with the number of classes in your dataset

    vgg_model= Model(inputs=base_model.input, outputs=predictions)
    
    vgg_model.compile(optimizer=Adam(lr=0.0001),  loss='sparse_categorical_crossentropy', metrics=['accuracy'])  # 编译模型
    vgg_model.fit(x_train, y_train, epochs=5, batch_size=64, validation_data=(x_test,y_test))  # 训练模型
    save_model(vgg_model, "/kaggle/working/vgg_model.h5")
    predictions= vgg_model.predict(x_test)
    y_pred = np.argmax(predictions, axis=1)

58889256/58889256 [==============================] - 0s 0us/step
Epoch 1/5
107/107 [==============================] - 82s 616ms/step - loss: 3.8495 - accuracy: 0.3334 - val_loss: 1.4178 - val_accuracy: 0.4017
Epoch 2/5
107/107 [==============================] - 58s 546ms/step - loss: 1.3217 - accuracy: 0.4566 - val_loss: 1.3941 - val_accuracy: 0.4275
Epoch 3/5
107/107 [==============================] - 59s 547ms/step - loss: 1.1668 - accuracy: 0.5272 - val_loss: 1.2972 - val_accuracy: 0.4650
Epoch 4/5
107/107 [==============================] - 58s 543ms/step - loss: 1.0464 - accuracy: 0.5809 - val_loss: 1.3039 - val_accuracy: 0.4525
Epoch 5/5
107/107 [==============================] - 58s 541ms/step - loss: 1.0023 - accuracy: 0.5987 - val_loss: 1.2711 - val_accuracy: 0.4942


/tmp/ipykernel_25/1695736743.py:24: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  save_model(vgg_model, "/kaggle/working/vgg_model.h5")


38/38 [==============================] - 9s 145ms/step


In [7]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, y_pred)
accuracy

0.49416666666666664

In [8]:
from collections import Counter

def find_majority_label(image_ids, labels):
    id_label_map = {}  # 创建一个空的字典来存储image_id和对应的label

    # 遍历image_ids和labels列表
    for image_id, label in zip(image_ids, labels):
        if image_id in id_label_map:
            id_label_map[image_id].append(label)
        else:
            id_label_map[image_id] = [label]

    majority_labels = {}  # 创建一个空的字典来存储每个唯一image_id的多数投票label

    # 对字典中的每个唯一image_id的label列表进行多数投票
    for image_id, label_list in id_label_map.items():
        label_counts = Counter(label_list)
        majority_label = label_counts.most_common(1)[0][0]
        majority_labels[image_id] = majority_label

    return majority_labels

y_votingpred=find_majority_label(img_ids_test, y_pred)
# 对图像 ID 进行去重
unique_img_ids = np.unique(img_ids_test)

# 根据去重后的图像 ID，获取对应的真实标签和预测标签
y_test_matched = []
y_test_matched = np.array([y_test[np.where(img_ids_test == img_id)[0][0]] for img_id in unique_img_ids])
y_votingpred_matched = np.array([y_votingpred[img_id] for img_id in unique_img_ids])

# 计算准确度
accuracy = accuracy_score(y_test_matched, y_votingpred_matched)
accuracy

0.5372340425531915

In [9]:
from sklearn.metrics import balanced_accuracy_score

# 计算平衡准确率
balanced_acc = balanced_accuracy_score(y_test_matched, y_votingpred_matched)

print("平衡准确率:", balanced_acc)

平衡准确率: 0.4547902336384498


In [10]:
from sklearn.metrics import confusion_matrix
print("-----Metrics Evaluation on Test Data-----")
print()
print("Confusion Matrix:\n",confusion_matrix(y_test_matched, y_votingpred_matched))
print()
#print("Classification Report:\n",classification_report(y_test_matched, y_votingpred_matched))

-----Metrics Evaluation on Test Data-----

Confusion Matrix:
 [[117   2   8  21   3]
 [ 14   6   0  13   1]
 [ 45   1  21  15   7]
 [ 18   1   1  41   2]
 [ 14   1   6   1  17]]



In [11]:
'''train_losses = []
train_accuracies = []
val_losses = []
val_accuracies = []

num_epochs = 100
for epoch in range(num_epochs):
    combine_model.train()
    train_loss = 0.0
    train_acc = 0.0

    for images, labels in train_loader:
        optimizer.zero_grad()
        images = images.to(device)
        labels = labels.to(device)

        #few steps are hidden related to model structure
        output = output.max(dim=1)[0]
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * images.size(0)
        _, predictions = torch.max(output, 1)
        train_acc += torch.sum(predictions == labels.data)

    train_loss = train_loss / len(train_loader.dataset)
    train_acc = train_acc.double() / len(train_loader.dataset)

    # Validation
    combine_model.eval()
    val_loss = 0.0
    val_acc = 0.0

    with torch.no_grad():
        for images, labels in val_loader:
            images = images.to(device)
            labels = labels.to(device)

            #few steps are hidden related to model structure
            output = output.max(dim=1)[0]
            loss = criterion(output, labels)

            val_loss += loss.item() * images.size(0)
            _, predictions = torch.max(output, 1)
            val_acc += torch.sum(predictions == labels.data)

        val_loss = val_loss / len(val_loader.dataset)
        val_acc = val_acc.double() / len(val_loader.dataset)

    # Append loss and accuracy to lists
    train_losses.append(train_loss)
    train_accuracies.append(train_acc.item())
    val_losses.append(val_loss)
    val_accuracies.append(val_acc.item())

    print(f"Epoch {epoch+1}/{num_epochs}")
    print(f"Train Loss: {train_loss:.4f} Train Acc: {train_acc:.4f}")
    print(f"Val Loss: {val_loss:.4f} Val Acc: {val_acc:.4f}")

# Plotting the loss and accuracy
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(range(1, num_epochs+1), train_losses, label='Train Loss')
plt.plot(range(1, num_epochs+1), val_losses, label='Validation Loss')
plt.xlabel('Epoch',fontweight='bold',fontsize=25)
plt.ylabel('Loss',fontweight='bold',fontsize=25)
plt.xticks(fontweight='bold',fontsize=25)
plt.yticks(fontweight='bold',fontsize=25)
# plt.title('Training and Validation Loss')
plt.legend()
plt.savefig("/root/breast/UBC/LossPlots.png",dpi=500)
plt.savefig("/root/breast/UBC/LossPlots.eps",dpi=500)
plt.subplot(1, 2, 2)
plt.plot(range(1, num_epochs+1), train_accuracies, label='Train Accuracy')
plt.plot(range(1, num_epochs+1), val_accuracies, label='Validation Accuracy')
plt.xlabel('Epoch',fontweight='bold',fontsize=25)
plt.ylabel('Accuracy',fontweight='bold',fontsize=25)
plt.xticks(fontweight='bold',fontsize=25)
plt.yticks(fontweight='bold',fontsize=25)
# plt.title('Training and Validation Accuracy')
plt.legend()
plt.tight_layout()
plt.savefig("/root/breast/UBC/AccPlots.png",dpi=500)
plt.savefig("/root/breast/UBC/AccPlots.eps",dpi=500)
plt.show()'''

'train_losses = []\ntrain_accuracies = []\nval_losses = []\nval_accuracies = []\n\nnum_epochs = 100\nfor epoch in range(num_epochs):\n    combine_model.train()\n    train_loss = 0.0\n    train_acc = 0.0\n\n    for images, labels in train_loader:\n        optimizer.zero_grad()\n        images = images.to(device)\n        labels = labels.to(device)\n\n        #few steps are hidden related to model structure\n        output = output.max(dim=1)[0]\n        loss = criterion(output, labels)\n        loss.backward()\n        optimizer.step()\n\n        train_loss += loss.item() * images.size(0)\n        _, predictions = torch.max(output, 1)\n        train_acc += torch.sum(predictions == labels.data)\n\n    train_loss = train_loss / len(train_loader.dataset)\n    train_acc = train_acc.double() / len(train_loader.dataset)\n\n    # Validation\n    combine_model.eval()\n    val_loss = 0.0\n    val_acc = 0.0\n\n    with torch.no_grad():\n        for images, labels in val_loader:\n            image